In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
indicator = 'positive'

In [ ]:
with open(f'time_series_{indicator}.pkl', 'rb') as f:
    data = pkl.load(f)
    
with open(f'indices_{indicator}.pkl', 'rb') as f:
    indices = pkl.load(f)

In [ ]:
data_size = data.shape[0]
window_size = 20
print(data_size, data.shape, window_size)

## Prompt GPT 4

In [ ]:
API_KEY = ''

In [ ]:
system_prompt = f"Your job is to act as a professional healthcare analyst. You will write a high-quality report that is informative and helps understand the current healthcare situation."
print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for i in indices:
    
    data_window = data[i:i+window_size]
    
    if indicator == 'positive':
        total_specimens = '|'.join([x for x in data_window[:,1]])
        total_a = '|'.join([x for x in data_window[:,2]])
        total_b = '|'.join([x for x in data_window[:,3]])
        pos_rate = '|'.join([str(f'{float(x):.2f}') for x in data_window[:,4]])
        a_rate = '|'.join([str(f'{float(x):.2f}') for x in data_window[:,5]])
        b_rate = '|'.join([str(f'{float(x):.2f}') for x in data_window[:,6]])
    
    elif indicator == 'mortality':
        inf_death = '|'.join([x for x in data_window[:,1]])
        pneum_death = '|'.join([x for x in data_window[:,2]])
        total_death = '|'.join([x for x in data_window[:,3]])
        ratio = '|'.join([str(f'{float(x):.2f}') for x in data_window[:,4]])
    
    if indicator == 'positive':
        user_prompt = f"Your task is to analyze the respiratory specimens testing positive for influenza over the last {window_size} weeks. "
        user_prompt += "The average ratio of positive speciemens is 6.26%."
        user_prompt += f"\n\nReview the time-series data provided for the last {window_size} weeks. "
        user_prompt += f"Each time-series consists of weekly values separated by a \'|\' token for the following indicators:\n"
        user_prompt += f"- Number of specimens tested: {total_specimens}\n"
        user_prompt += f"- Number of positive specimens for Influenza A: {total_a}\n"
        user_prompt += f"- Number of positive specimens for Influenza B: {total_b}\n"
        user_prompt += f"- Ratio of positive specimens (%): {pos_rate}\n"
        user_prompt += f"- Ratio of positive specimens for Influenza A (%): {a_rate}\n"
        user_prompt += f"- Ratio of positive specimens for Influenza B (%): {b_rate}\n\n"
    
    elif indicator == 'mortality':
        user_prompt = f"Your task is to analyze the mortality from Influenza or Pneumonia over the last {window_size} weeks. "
        user_prompt += "The average ratio of mortality from Influenza or Pneumonia to the total number of death is 7.84%."
        user_prompt += f"\n\nReview the time-series data provided for the last {window_size} weeks. "
        user_prompt += f"Each time-series consists of weekly values separated by a \'|\' token for the following indicators:\n"
        user_prompt += f"- Total number of death: {total_death}\n"
        user_prompt += f"- Number of death from influenza: {inf_death}\n"
        user_prompt += f"- Number of death from pneumonia: {pneum_death}\n"
        user_prompt += f"- Ratio of mortality from Influenza or Pneumonia (%): {ratio}\n\n"
        
    user_prompt += f"Based on this time-series data, write a concise report that provides insights crucial for understanding the current healthcare situation. "
    user_prompt += f"Your report should be limited to five sentences, yet comprehensive, highlighting key trends and considering their potential impact on the healthcare system. "
    user_prompt += f"Do not write redundant information."
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content
    
    with open(f'gpt_summary/{i}_{indicator}.txt', 'w') as f:
        f.write(text)